# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [45]:
%pylab inline
import pickle
from os import listdir, path, chdir
import numpy as np
from sklearn import svm, metrics
import io

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [46]:
chdir(r'C:\Users\Chiara\Desktop\RoboCup\GitHub\programming-humanoid-robot-in-python\joint_control')
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [47]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE

    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    f = io.open(filename,'rb')
    data = pickle.load(f)
    target = [i] * len(data)

    return data, target

In [48]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE

for c in range(len(classes)):
    
    c_data, c_target = load_pose_data(c)

    #absolutely shitty code need to find a better substitute
    
    for entry in range(len(c_data)):
        
        all_data.append(c_data[entry])
        all_target.append(c_target[entry])

all_data = np.array(all_data)
all_target = np.array(all_target)
print 'total number of data', len(all_data)

total number of data 222


In [60]:
# shuffle data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)

training_index = permutation[:n_training_data]
test_index = permutation[n_training_data:]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [62]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning
        

In [63]:
# YOUR CODE HERE
data_training = all_data[training_index]
target_training = all_target[training_index]

data_test = all_data[test_index]
target_test = all_target[test_index]
clf.fit(data_array, target_array)



SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [52]:
def evaluate(expected, predicted):
    
    
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [64]:
expected = target_test
predicted = clf.predict(data_test)


evaluate(expected, predicted)  #as we can see, we get very good results

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         8
           1       1.00      1.00      1.00         2
           2       0.90      1.00      0.95         9
           3       1.00      1.00      1.00         4
           4       1.00      1.00      1.00         6
           5       1.00      1.00      1.00         3
           6       1.00      1.00      1.00         8
           7       1.00      0.75      0.86         4
           8       1.00      1.00      1.00         9
           9       1.00      1.00      1.00         2
          10       1.00      1.00      1.00        12

   micro avg       0.99      0.99      0.99        67
   macro avg       0.99      0.98      0.98        67
weighted avg       0.99      0.99      0.98        67


Confusion matrix:
[[ 8  0  0  0  0  0  0  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0  0  0]
 [ 0  0  9  0  0  0  0  0  0  0  0]
 [ 0  0  0  4  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [65]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [66]:
clf2 = pickle.load(io.open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), array([10]))